## Prepare the latest sales record
6 months sales record history to reset the inventory aging time.
The guilty inventory count starts from the month of no sales.

1.  Delete the blank rows of the excel to have the header as 1st row.
2.  Save the excel by csv

In [2]:
import logging
import sqlite3

import pandas as pd

logging.basicConfig(
    level=logging.DEBUG, format=" %(asctime)s -  %(levelname)s -  %(message)s"
)

In [3]:
def create_tables():
    # Connect to sqlite3
    try:
        conn = sqlite3.connect("guilty_inventory.sqlite3")
        c = conn.cursor()

        # Create tables
        # t_sales table
        c.execute(
            """
        create table if not exists t_sales (sales_month text, sales_model text,
        sales_qty int, sales_amt float, primary key(sales_month, sales_model))
        """
        )

        # t_purchase table
        c.execute(
            """
        create table if not exists t_purchase (purchase_month text, purchase_model text,
        purchase_qty int, purchase_amt float, primary key(purchase_month, purchase_model))
        """
        )

        # t_inv table
        c.execute(
            """
        create table if not exists t_inv (inv_month text, inv_model text, inv_cat int,
        inv_qty int, inv_amt float, inv_age int, primary key(inv_month, inv_model, inv_cat, inv_age))
        """
        )

        # t_business_unit table
        c.execute(
            """
        create table if not exists t_business_unit (prod_category int primary key,
        business_unit text, bu_description text)
        """
        )

        # t_rate table
        c.execute(
            """
        create table if not exists t_rate (rate_month text primary key,
        rate_yen float)
        """
        )

        # t_aging table
        c.execute(
            """
        create table if not exists t_aging (
        actual_age, n0_age, n1_age, n2_age, n3_age, n4_age, n5_age, n6_age,
        n7_age, n8_age, n9_age, n10_age, n11_age, n12_age, n13_age
        )
        """
        )

        # t_guilty_base
        c.execute(
            """
        create table if not exists t_guilty_base (
        inv_month text, inv_model text, inv_cat int, business_unit text,
        inv_age int, inv_age_mod int, rate_yen float, inv_qty int, inv_KMXN float,
        inv_MYEN float, sales_n0_Q int, sales_n0_KMXN float,
        sales_n0_MYEN float, sales_n1_Q int, sales_n1_KMXN float,
        sales_n1_MYEN float, sales_n2_Q int, sales_n2_KMXN float,
        sales_n2_MYEN float, sales_n3_Q int, sales_n3_KMXN float,
        sales_n3_MYEN float, sales_n4_Q int, sales_n4_KMXN float,
        sales_n4_MYEN float, sales_n5_Q int, sales_n5_KMXN float,
        sales_n5_MYEN float, sales_n6_Q int, sales_n6_KMXN float,
        sales_n6_MYEN float, purchase_n0_Q int, purchase_n1_Q int,
        purchase_n2_Q int, purchase_n3_Q int, purchase_n4_Q int,
        purchase_n5_Q int, purchase_n6_Q int, primary key(inv_month, inv_model, inv_age))
        """
        )

        # t_guilty_list
        c.execute(
            """
        create table if not exists t_guilty_list (
        inv_month text, inv_model text, inv_model_digest text, bussines_unit text,
        inv_age int, inv_qty int, inv_KMXN float, inv_MYEN float, sales_n0_Q int,
        sales_n0_KMXN float, sales_n1_Q int, sales_n1_KMXN float, sales_n2_Q int,
        sales_n2_KMXN float, sales_n3_Q int, sales_n3_KMXN float, sales_n4_Q int,
        sales_n4_KMXN float, sales_n5_Q int, sales_n5_KMXN float, sales_n6_Q int,
        sales_n6_KMXN float, primary key(inv_month, inv_model, inv_age))
        """
        )

        # t_guilty_digest
        c.execute(
            """
        create table if not exists t_guilty_digest (
        inv_month text, inv_model_digest text, bussines_unit text, inv_qty int,
        inv_KMXN float, inv_n1_Q int, sales_n0_Q int, sales_n0_KMXN float,
        sales_n1_Q int, sales_n1_KMXN float, sales_n2_Q int, sales_n2_KMXN float,
        sales_n3_Q int, sales_n3_KMXN float, sales_n4_Q int, sales_n4_KMXN float,
        sales_n5_Q int, sales_n5_KMXN float,sales_n6_Q int, sales_n6_KMXN float,
        primary key(inv_month, inv_model_digest))
        """
        )

        # Close connection to sqlite3
        c.close()
        conn.close()

    except Exception as e:
        print(e)

    else:
        print("Successfully finished without error")


create_tables()

Successfully finished without error


In [12]:
def t_sales_update_csv(targetFile):
    # Sales record preparation from csv and add to sqlite3 t_sales table
    # Copy the original sheet to a new sheet & paste value and save as csv.
    # Billing doc. date must be formatted to date (ex: 2020/8/2)
    # Sort by Billing doc. is ideal.

    useCols = ["Billing doc. date", "Material", "Net Qty", "Net $"]

    df = pd.read_csv(targetFile, usecols=useCols, parse_dates=[0])
    df = df.rename(
        columns={
            "Billing doc. date": "sales_month",
            "Material": "sales_model",
            "Net Qty": "sales_qty",
            "Net $": "sales_amt",
        }
    )
    
    # Modify the sales month to have 1st date of the month
    df["sales_month"] = df["sales_month"].apply(lambda x: x.replace(day=1))
    grouped = df.groupby(["sales_month", "sales_model"], as_index=False)
    # print(grouped.sum())

    # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add grouped.sum() into t_sales table
        grouped.sum().to_sql("t_sales", conn, if_exists="append", index=None)
    except Exception as e:
        print(e)
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_sales_update_csv("sales-201909-202008.csv")

Done with no error


In [13]:
def t_inv_update_csv(targetFile):
    # Inventory aging preparation from csv and add to sqlite3 t_inv table
    # Copy the original sheet to a new sheet & paste value and save as csv.
    # Add columun "inv_month" with date format like 2019/9/1 (day must be 1st day)

    useCols = ["CatNo", "Material", "Fecha", "Suma de Total Qty", "Suma de Total $", "inv_month"]

    df = pd.read_csv(targetFile, usecols=useCols, parse_dates=[5])
    df = df.rename(
        columns={
            "CatNo": "inv_cat",
            "Material": "inv_model",
            "Fecha": "inv_age",
            "Suma de Total Qty": "inv_qty",
            "Suma de Total $": "inv_amt"
        }
    )
    
    
    df = df.reindex(columns=['inv_month', 'inv_model', 'inv_cat', 'inv_qty', 'inv_amt', 'inv_age'])
    
    # Replace the aging date
    agedict = {'30D': 30, '60D': 60, '90D': 90, '180D': 180, '270D': 270, '360D': 360, 'MAS': 999}
    df = df.replace(agedict)
  
    # Modify the inv_month to have 1st date of the month, just in case.
    df["inv_month"] = df["inv_month"].apply(lambda x: x.replace(day=1))
    
    # logging.debug('df status :\n' + str(df))
    # logging.debug('df is' + str(df['inv_month'].dtype))

    # Grouping by in_month and inv_model
    grouped = df.groupby(["inv_month", "inv_model", "inv_cat", "inv_age"], as_index=False)
    # logging.debug('grouped.sum() status:\n' + str(grouped.sum()))

     # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add grouped.sum() into t_inv table
        grouped.sum().to_sql("t_inv", conn, if_exists="append", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_inv_update_csv("inv201909-202008.csv")

Done with no error


In [14]:
def t_purchase_update_csv(targetFile):
    # Purchase preparation from csv and add to sqlite3 t_purchase table
    # Copy the original sheet to a new sheet & paste value and save as csv.

    useCols = ["Material", "Suma de Total Qty", "Suma de Total $", "Mes de arribo"]

    df = pd.read_csv(targetFile, usecols=useCols, parse_dates=[3])
    df = df.rename(
        columns={
            "Material": "purchase_model",
            "Suma de Total Qty": "purchase_qty",
            "Suma de Total $": "purchase_amt",
            "Mes de arribo": "purchase_month"
        }
    )
    
    
    df = df.reindex(columns=['purchase_month', 'purchase_model', 'purchase_qty', 'purchase_amt'])
    
    # Modify the purchase_month to have 1st date of the month, just in case.
    df["purchase_month"] = df["purchase_month"].apply(lambda x: x.replace(day=1))
    
#     logging.debug('df status :\n' + str(df))
#     logging.debug('df is' + str(df['purchase_month'].dtype))

    # Grouping by in_month and inv_model
    grouped = df.groupby(["purchase_month", "purchase_model"], as_index=False)
#   logging.debug('grouped.sum() status:\n' + str(grouped.sum()))

     # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add grouped.sum() into t_purchase table
        grouped.sum().to_sql("t_purchase", conn, if_exists="append", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_purchase_update_csv("purchasetest.csv")

Done with no error


In [15]:
def t_business_unit_update_csv(targetFile):
    # business_unit preparation from csv and add to sqlite3 t_business_unit table

    df = pd.read_csv(targetFile)
    
    # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add business unit into t_business_unit table
        df.to_sql("t_business_unit", conn, if_exists="replace", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_business_unit_update_csv("business_unit.csv")

Done with no error


In [16]:
def t_rate_update_csv(targetFile):
    # Rate table preparation from csv and add to sqlite3 t_rate table

    df = pd.read_csv(targetFile, parse_dates=[0])
        
    # Modify the inv_month to have 1st date of the month, just in case.
    df["rate_month"] = df["rate_month"].apply(lambda x: x.replace(day=1))

    
    # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add rate into t_rate table
        df.to_sql("t_rate", conn, if_exists="replace", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_rate_update_csv("rate.csv")

Done with no error


In [19]:
def t_guilty_base_generate():

    try:
        # Connect to sqlite3
        conn = sqlite3.connect("guilty_inventory.sqlite3")
        c = conn.cursor()

        # Clear data in t_guilty_base
        c.execute(
            """
        delete from t_guilty_base
        """
        )
        # Add data into t_guilty_base
        c.execute(
            """
        insert into t_guilty_base
        SELECT 
        inv_month,
        inv_model,
        inv_cat,
        business_unit,
        inv_age,
        inv_age as inv_age_mod,
        rate_yen,
        inv_qty,
        inv_amt / 1000 as inv_KMXN,
        inv_amt * rate_yen as inv_MYEN,

        sn0.sales_qty as sales_n0_Q,
        sn0.sales_amt / 1000 as sales_n0_KMXN,
        sn0.sales_amt * rate_yen / 1000000 as sales_n0_MYEN,

        sn1.sales_qty as sales_n1_Q,
        sn1.sales_amt / 1000 as sales_n1_KMXN,
        sn1.sales_amt * rate_yen / 1000000 as sales_n1_MYEN,

        sn2.sales_qty as sales_n2_Q,
        sn2.sales_amt / 1000 as sales_n2_KMXN,
        sn2.sales_amt * rate_yen / 1000000 as sales_n2_MYEN,

        sn3.sales_qty as sales_n3_Q,
        sn3.sales_amt / 1000 as sales_n3_KMXN,
        sn3.sales_amt * rate_yen / 1000000 as sales_n3_MYEN,

        sn4.sales_qty as sales_n4_Q,
        sn4.sales_amt / 1000 as sales_n4_KMXN,
        sn4.sales_amt * rate_yen / 1000000 as sales_n4_MYEN,

        sn5.sales_qty as sales_n5_Q,
        sn5.sales_amt / 1000 as sales_n5_KMXN,
        sn5.sales_amt * rate_yen / 1000000 as sales_n5_MYEN,

        sn6.sales_qty as sales_n6_Q,
        sn6.sales_amt / 1000 as sales_n6_KMXN,
        sn6.sales_amt * rate_yen / 1000000 as sales_n6_MYEN,

        pn0.purchase_qty as purchase_n0_Q,
        pn1.purchase_qty as purchase_n1_Q,
        pn2.purchase_qty as purchase_n2_Q,
        pn3.purchase_qty as purchase_n3_Q,
        pn4.purchase_qty as purchase_n4_Q,
        pn5.purchase_qty as purchase_n5_Q,
        pn6.purchase_qty as purchase_n6_Q

        FROM t_inv 

        left outer join
        t_business_unit
        on inv_cat = prod_category
        
        left outer join
        t_rate
        on inv_month = rate_month

        left outer join
        t_sales sn0
        on inv_month = sn0.sales_month and inv_model = sn0.sales_model

        left outer join
        t_sales sn1
        on datetime(inv_month, '-1 months') = sn1.sales_month and inv_model = sn1.sales_model

        left outer join
        t_sales sn2
        on datetime(inv_month, '-2 months') = sn2.sales_month and inv_model = sn2.sales_model

        left outer join
        t_sales sn3
        on datetime(inv_month, '-3 months') = sn3.sales_month and inv_model = sn3.sales_model

        left outer join
        t_sales sn4
        on datetime(inv_month, '-4 months') = sn4.sales_month and inv_model = sn4.sales_model

        left outer join
        t_sales sn5
        on datetime(inv_month, '-5 months') = sn5.sales_month and inv_model = sn5.sales_model

        left outer join
        t_sales sn6
        on datetime(inv_month, '-6 months') = sn6.sales_month and inv_model = sn6.sales_model

        left outer join
        t_purchase pn0
        on inv_month = pn0.purchase_month and inv_model = pn0.purchase_model

        left outer join
        t_purchase pn1
        on datetime(inv_month, '-1 months') = pn1.purchase_month and inv_model = pn1.purchase_model

        left outer join
        t_purchase pn2
        on datetime(inv_month, '-2 months') = pn2.purchase_month and inv_model = pn2.purchase_model

        left outer join
        t_purchase pn3
        on datetime(inv_month, '-3 months') = pn3.purchase_month and inv_model = pn3.purchase_model

        left outer join
        t_purchase pn4
        on datetime(inv_month, '-4 months') = pn4.purchase_month and inv_model = pn4.purchase_model

        left outer join
        t_purchase pn5
        on datetime(inv_month, '-5 months') = pn5.purchase_month and inv_model = pn5.purchase_model

        left outer join
        t_purchase pn6
        on datetime(inv_month, '-6 months') = pn6.purchase_month and inv_model = pn6.purchase_model

        """
        )

        # Commit
        conn.commit()

        # Close connection to sqlite3
        c.close()
        conn.close()

    except Exception as e:
        print(e)

    else:
        print("Successfully finished without error")


t_guilty_base_generate()

Successfully finished without error


In [ ]:
def t_guilty_base_update():
    try:
        # Connect to sqlite3
        conn = sqlite3.connect("guilty_inventory.sqlite3")
        c = conn.cursor()

        # Update inv_age_mod of t_guilty_base according to latest purchase and sales.
        c.execute(
            "update t_guilty_base set inv_age_mod = 30 where sales_n0_Q > 0 or purchase_n0_Q > 0"
        )

        # Commit
        conn.commit()

        # Close connection to sqlite3
        c.close()
        conn.close()

    except Exception as e:
        print(e)

    else:
        print("Successfully finished without error")


t_guilty_base_update()